# Evitement de collisions - Entrainement du modèle (ResNet18)

Dans ce notebook, nous allons entrainer notre modèle afin qu'il puisse classifier les images capturées avec la caméra dans les classes ``libre`` ou ``bloquer``. Le modèle que nous allons utiliser est le modèle Restnet18.
Nous allons utiliser la librairie Keras / Tensorflow pour créer notre modèle.

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

### Téléchargement et extraction des données collectées 

Tout d'abord il faut récupérer les données collectées contenues dans le fichier ``dataset.zip`` que nous avons créé précedemment. Pour cela, il suffit d'exécuter la commande ci-dessous.

In [ ]:
#!unzip -q dataset.zip

Un dossier nommé ``dataset`` devrait apparaître dans l'explorateur de fichiers.

# Création des datasets

### Chargement des images dans le dataset d'entrainement

A partir des images sauvegardées dans le répertoire de travail, on peut maintenant créer notre dataset. On commence par créer une liste contenant les deux classes utilisées :

In [ ]:
CATEGORIES = ['bloquer', 'libre']
datasets = {}
for name in CATEGORIES:
    datasets[name] = []

On charge ensuite les données à l'aide de la fonction ``image_dataset_from_directory`` de Keras, en précisant la résolution des images et le type de label associé (ici on utilise le type ``categorical`` - voir ci-dessous). On demande également d'utiliser un ``batch size`` de 1 et d'utiliser 90% des images pour l'entrainement et les 10% restant pour les validations :

In [ ]:
# Récupère le répertoire courant
repertoire_courant = os.getcwd()

# Création du dataset d'entrainement
dataset_entrainement = tf.keras.preprocessing.image_dataset_from_directory(
    repertoire_courant+"/dataset/",
    validation_split=0.2,
    subset="training",
    image_size=(224, 224),
    batch_size=1,
    shuffle=True,
    seed=123,
    label_mode='categorical')

# Création du dataset de validation
dataset_validation = tf.keras.preprocessing.image_dataset_from_directory(
    repertoire_courant+"/dataset/",
    validation_split=0.2,
    subset="validation",
    image_size=(224, 224),
    batch_size=1,
    shuffle=True,
    seed=123,
    label_mode='categorical')

Observons le nom des classes utilisées :

In [ ]:
class_names = dataset_entrainement.class_names
print(class_names)

Regardons le format du tenseur contenu dans le dataset :

In [ ]:
for image,label in dataset_entrainement.take(2):
    print(image.shape)
    print(label.shape)

Regardons comment est codée une image :

In [ ]:
for image,label in dataset_entrainement.take(1):
    print(image[0])

Affichons quelques labels codé de manière "categorical" et leur valeur équivalente "binaire" :

In [ ]:
for image,label in dataset_entrainement.take(5):
    print("Label categorical : %s" %label[0])
    print("Label binaire correspondant : %s" %np.argmax(label[0], axis=None, out=None))
    print("Classe correspondante : %s" %class_names[np.argmax(label[0], axis=None, out=None)])
    print("")

Affichons maintenant quelques images :

In [ ]:
iterator = iter(dataset_entrainement)

plt.figure(figsize=(10, 10))
for i in range(8):
    ax = plt.subplot(4, 4, i + 1)
    image, label = iterator.get_next()
    plt.imshow(image[0].numpy().astype("uint8"))
    plt.title(class_names[np.argmax(label[0], axis=None, out=None)])
    plt.axis("off")

### Traitement des images

Pour le Resnet18 avec Keras / Tensorflow, il n'y a aucune action de prétraitement à effectuer !

# Création du modèle

Pour utiliser le modèle RestNet18, nous allons utiliser le package Image-classifiers disponnible sur le github : https://github.com/AlexandreBourrieau/classification_models

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

In [ ]:
# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False,pooling=False)

model = tf.keras.models.Model(inputs=base_model.input, outputs=base_model.output)
model.summary()

In [ ]:
# Désactivation des couches pour l'entrainement
for layer in base_model.layers:
    layer.trainable = False
model.summary()

On ajoute ensuite la couche d'applatissemnt des sorties et la couche dense avec 2 neurones (1 neurone par classe) :

In [ ]:
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation='softmax')(x)

# Création du modèle global
model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

# Affichage des informations sur le modèle
model.summary()

In [ ]:
# Liste des couches du modèle
for i, layer in enumerate(model.layers):
   print(i, layer.name)

On sélectionne éventuellement les couches à entrainer :

In [ ]:
#68 stage4_unit1_conv1
#72 stage4_unit1_conv2
#78 stage4_unit2_conv1
#82 stage4_unit2_conv2
#87 dense_1

for layer in model.layers[87:]:
   layer.trainable = True
model.summary()

# Entrainement du modèle

Maintenant, on lance l'entrainement du modèle. On fait également en sorte de sauvegarder le meilleur modèle en considérant les résultats obtenus sur les données de validation.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Nombre de périodes d'entrainement
periodes = 30

# Définition de la fonction d'enregistrement automatique du meilleur modèle
model_save = ModelCheckpoint('meilleur_modele.hdf5', save_best_only=True, monitor='val_loss', mode='min')

# Définition de l'ooptimiseur
adam = keras.optimizers.Adam(learning_rate=1e-4)

# Entrainement du modèle : from_logits=True car on utilise un Softmax en sortie de notre modèle
model.compile(optimizer=adam, loss='categorical_crossentropy')
historique = model.fit(dataset_entrainement,validation_data=dataset_validation,verbose=1,epochs=periodes, callbacks=[model_save])

Observons pour finir l'évolution de la qualité de l'apprentissage du modèle :

In [ ]:
loss = historique.history['loss']
val_loss = historique.history['val_loss']

intervalle_periodes = range(periodes)

plt.plot(intervalle_periodes, loss, label="Erreur d'entrainement")
plt.plot(intervalle_periodes, val_loss, label="Erreur de validation")
plt.legend(loc='upper right')
plt.title("Erreur d'entrainement et de validation")
plt.show()